# Component

> TODO fill in description

In [1]:
#| default_exp component

In [2]:
#| hide
from nbdev.showdoc import *

In [20]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import asyncio
from abc import ABC, abstractmethod
from typing import Type

from fbdev import BasePort, BaseInputPort, InputPort, BaseOutputPort, OutputPort

In [5]:
#|export
__all__ = ['Component']

In [ ]:
#|export
class Expando:
    def __init__(self):
        self.idx = ()
        self._attrs = {}
        
    def __getattr__(self, key):
        return self[key]
        
    def __getitem__(self, key):
        if key in self._attrs.keys():
            return self._attrs[key]
        else:
            #TODO proper exception in Expando.__getitem__
            raise Exception(f"{key} does not exist")
        
    def keys(self):
        return self._attrs.keys()
    
    def values(self):
        return self._attrs.values()
        
    def __iter__(self):
        return self._attrs.__iter__()
    
    def __contains__(self, key):
        return key in self._attrs

    def __len__(self):
        return self._attrs.__len__()
    
    def __str__(self):
        return ", ".join([f"{k}: {v}" for k,v in self._ports.items()])

In [6]:
#|export
class ComponentPorts(Expando):
    def __init__(self):
        self.idx = ()
        self._ports = {}
        
    def _add(self,
             port:BasePort):
        if port.name in self._ports:
            #TODO proper exception in PortContainer._add
            raise Exception(f"Port with name {port.name} already exists.")
        self.idx = tuple(list(self.idx) + [port])
        self._attrs[port.name] = port

In [ ]:
#|export
class ComponentConfig(Expando):
    pass

In [6]:
#|export
class Component(ABC):
    config_input_specs = [] #TODO the way these specs look like are a bit ugly
    input_specs = []
    output_specs = []
    
    def __init__(self, name):
        self.name = name
        self.config = ComponentConfig()
        self._is_configured = True # Will be set to False if the check fails below
        self.config_inputs = ComponentPorts()
        self.inputs = ComponentPorts()
        self.outputs = ComponentPorts()
        
        def attach_port(self, port, port_container):
            port_container._add(port)
            port.attach(self, port_container.idx.index(port))
                    
        for port_spec in self.config_input_specs:
            name, port_kwargs = port_spec
            if 'default' in port_kwargs: self.config._attrs[name] = port_kwargs['default']
            else: self._is_configured = False
            attach_port(InputPort(name, **port_kwargs), self.config_inputs)
            
        for port_spec in self.input_specs:
            name, port_type, port_kwargs = port_spec
            attach_port(port_type(name, **port_kwargs), self.inputs)

        for port_spec in self.output_specs:
            name, port_type, port_kwargs = port_spec
            attach_port(port_type(name, **port_kwargs), self.outputs)
    
    def is_configured(self):
        return self._is_configured
    
    async def await_initial_config(self):
        for config_name, config_input_port in self.config_inputs:
            if not config_name in self.config:
                self.config._attrs[config_name] = await self.config_inputs[config_name].get()
        self._is_configured = True
    
    async def update_config(self):
        for config_name, input_port in self.config_inputs:
            if input_port.is_filled():
                self.config._attrs[config_name] = await input_port.get()
    
    @abstractmethod
    async def run(self):
        pass